In [1]:
!pip install torchinfo
!pip install optuna

import optuna
import glob
import re
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import copy
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchinfo import summary
from PIL import Image
from collections import Counter
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True


In [2]:
!git clone https://github.com/Bjarten/early-stopping-pytorch/
%cd ./early-stopping-pytorch/
!python pytorchtools.py install
from pytorchtools import EarlyStopping

Cloning into 'early-stopping-pytorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.
/kaggle/working/early-stopping-pytorch


In [3]:
%cd ../

/kaggle/working


In [4]:
def img_files(path):  
  img=[]
  tag=[]
  for file in sorted(glob.glob(path)):
    img.append(file)
    s=re.match("^.*?-",file).group()
    if s in tag:
      continue
    else:
      tag.append(s)

  
  return img,tag

In [5]:
def reset_weights(m):
    if isinstance(m, nn.LSTM) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [6]:
def preprocess_data(data):
  l=[]
  data=np.reshape(np.array(data),(-1,20))
  for i in data:
    s=re.search("(?<=-).?",i[0])
    if s.group()=='N':
      l.append(0)
    else:
      l.append(1)
  
  label=np.array(l,dtype=np.float32)

  return data,label

In [7]:
class RoboDataset(Dataset):
  def __init__(self, file,labels,transform):
    self.imagefiles=file
    self.labels=labels
    self.transform=transform
  def __len__(self):
    return len(self.imagefiles)
  def __getitem__(self,idx):
    l=[]
    for i in self.imagefiles[idx]:
      img=Image.open(i)
      l.append(img)
    frame=[]
    for j in l:
      j=self.transform(j)
      frame.append(j)
    frame=torch.stack(frame)

    return frame,self.labels[idx]

In [8]:
class LSTM(nn.Module):
  def __init__(self,hidden_size,num_layers,num_classes):
    super(LSTM, self).__init__()
    
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.num_classes=num_classes
    self.backbone = torch.hub.load('/kaggle/working/trainval/yolov5/yolov5/', 'custom', source='local', path='/kaggle/working/trainval/yolov5n_backbone.pt', force_reload=True, autoshape=False)
    features=256
    for param in self.backbone.parameters():
      param.requires_grad = False
    self.pool=nn.AdaptiveAvgPool2d(1)
    self.flat=nn.Flatten()
    self.lstm = nn.LSTM(input_size=features, hidden_size=hidden_size,num_layers=num_layers, batch_first=True)    
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    batch, seq, C,H, W = x.size()
    c_in = x.view(batch * seq, C, H, W)
    bone_out = self.backbone(c_in)
    pool_out=self.pool(bone_out)
    flat_out=torch.flatten(pool_out)
    r_out, (h_n, h_c) = self.lstm(flat_out.view(batch,seq,-1))
    out = r_out[:, -1, :]
    out=self.fc(out)
    return out

In [9]:
def train_model(modelo,earlystop,epochs,trainloader,validloader,optimizer,criterion):

  early_stopping = EarlyStopping(patience=earlystop)
  history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
  for i in range(1,epochs):
    modelo.train()
    t_loss=0
    t_correct=0
    t_total=0
  
    for j,(inputs,labels) in enumerate(trainloader):
      inputs=inputs.to(device)
      labels=labels.unsqueeze(1).to(device)
      
      optimizer.zero_grad()
      outputs=modelo(inputs)
      loss=criterion(outputs,labels)
      pred=(outputs>0.0).float()

      loss.backward()
      optimizer.step()

      t_loss += loss.item()
      t_correct+=(pred==labels).sum().item()
      t_total+=labels.size(0)

    train_loss=t_loss/len(trainloader)
    tr_acc=100.*t_correct/t_total


    print("Epoch: %d Training, loss: %1.5f, correct/total: %d / %d, accuracy:%1.3f " % (i, train_loss,t_correct,t_total,tr_acc))
      
    modelo.eval()
    v_loss=0
    v_correct=0
    v_total=0

    with torch.no_grad():
      for inputs,labels in validloader:
        inputs=inputs.to(device)
        labels=labels.unsqueeze(1).to(device) 
      
        outputs=modelo(inputs)

        loss=criterion(outputs,labels)
        pred=(outputs>0.0).float()

        v_correct+=(pred==labels).sum().item()
        v_loss += loss.item()
        v_total+=labels.size(0)

    val_loss=v_loss/len(validloader)
    v_acc=100.*v_correct/v_total


        
    print("Epoch: %d Validation, loss: %1.5f, correct/total: %d / %d, accuracy:%1.3f " % (i, val_loss,v_correct,v_total,v_acc))
    history['train_loss'].append(train_loss)
    history['test_loss'].append(val_loss)
    history['train_acc'].append(tr_acc)
    history['test_acc'].append(v_acc)
    early_stopping(val_loss, modelo)
          
    if early_stopping.early_stop:
        print("Early stopping")
        break
        
  return modelo,history

In [10]:
from distutils.dir_util import copy_tree
copy_tree('/kaggle/input/', '/kaggle/working')

['/kaggle/working/trainval/trainval/trainval/V275-R0761.png',
 '/kaggle/working/trainval/trainval/trainval/V685-R5045.png',
 '/kaggle/working/trainval/trainval/trainval/V340-N20351.png',
 '/kaggle/working/trainval/trainval/trainval/V104-N2101.png',
 '/kaggle/working/trainval/trainval/trainval/V850-N0301.png',
 '/kaggle/working/trainval/trainval/trainval/V295-R1140.png',
 '/kaggle/working/trainval/trainval/trainval/V191-R0476.png',
 '/kaggle/working/trainval/trainval/trainval/V334-N0610.png',
 '/kaggle/working/trainval/trainval/trainval/V691-R0490.png',
 '/kaggle/working/trainval/trainval/trainval/V478-N1241.png',
 '/kaggle/working/trainval/trainval/trainval/V346-N0721.png',
 '/kaggle/working/trainval/trainval/trainval/V810-N0276.png',
 '/kaggle/working/trainval/trainval/trainval/V397-R0685.png',
 '/kaggle/working/trainval/trainval/trainval/V736-N0121.png',
 '/kaggle/working/trainval/trainval/trainval/V552-N0276.png',
 '/kaggle/working/trainval/trainval/trainval/V143-R3800.png',
 '/kagg

In [11]:
train_files,tags=img_files("/kaggle/working/trainval/trainval/trainval/*.png")
trainX,trainY=preprocess_data(train_files)

In [12]:
train_transform=transforms.Compose([
      transforms.Resize((320,320)),
      transforms.ToTensor(),  
      transforms.Normalize([0.4335, 0.4272, 0.4201], [0.2497, 0.2502, 0.2524]),
                            
])

valid_transform=transforms.Compose([
      transforms.Resize((320,320)),
      transforms.ToTensor(),  
      transforms.Normalize([0.4335, 0.4272, 0.4201], [0.2497, 0.2502, 0.2524]),
                           
])

In [13]:
def hyptuning(version):
    param={"num_layers":version.suggest_int("num_layer",1,4),
           "hidden_size":version.suggest_int("hidden_size",16,128),
           "learning_rate":version.suggest_uniform("learning_rate",5e-6,1e-2),
           

        
    }
    epochs=30
    hidden_size=param["hidden_size"]
    num_layers=param["num_layers"]
    num_classes=1
    learning_rate=param["learning_rate"]
    earlystop=7

    modelo = LSTM(hidden_size, num_layers,num_classes)
    modelo.to(device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(modelo.parameters(), lr=param["learning_rate"])
    prueba=RoboDataset(trainX,trainY,transform=train_transform)
    nfold=8
    kfold=KFold(n_splits=nfold,shuffle=True)
    foldperf={}
    for fold,(train_idx,test_idx) in enumerate(kfold.split(prueba)):
        print('Fold No. {}'.format(fold+1))
        train_sampler=SubsetRandomSampler(train_idx)
        test_sampler=SubsetRandomSampler(test_idx)
        trainloader=DataLoader(prueba,batch_size=32,sampler=train_sampler)
        validloader=DataLoader(prueba,batch_size=32,sampler=test_sampler)
        modelo,history=train_model(modelo,earlystop,epochs,trainloader,validloader,optimizer,criterion)
        foldperf['fold{}'.format(fold+1)] = history  
        modelo.apply(reset_weights)
    
    testloss_fold=[]
    k=8
    for f in range(k):
        testloss_fold.append(np.min(foldperf['fold{}'.format(f+1)]['test_loss']))
            

    return np.mean(testloss_fold)

In [14]:
study=optuna.create_study(direction="minimize")
study.optimize(hyptuning,n_trials=3)
print("Mejor resultado")
versions=study.best_trial
print(versions.values)
print(versions.params)

[I 2022-05-27 17:00:22,180] A new study created in memory with name: no-name-cd6684ea-0f8d-4962-b0a3-e24949ac2a23


Fold No. 1
Epoch: 1 Training, loss: 0.67755, correct/total: 412 / 692, accuracy:59.538 
Epoch: 1 Validation, loss: 0.75364, correct/total: 47 / 99, accuracy:47.475 
Epoch: 2 Training, loss: 0.66598, correct/total: 417 / 692, accuracy:60.260 
Epoch: 2 Validation, loss: 0.76324, correct/total: 47 / 99, accuracy:47.475 
EarlyStopping counter: 1 out of 7
Epoch: 3 Training, loss: 0.62761, correct/total: 414 / 692, accuracy:59.827 
Epoch: 3 Validation, loss: 0.60078, correct/total: 69 / 99, accuracy:69.697 
Epoch: 4 Training, loss: 0.60850, correct/total: 470 / 692, accuracy:67.919 
Epoch: 4 Validation, loss: 0.54247, correct/total: 62 / 99, accuracy:62.626 
Epoch: 5 Training, loss: 0.59585, correct/total: 477 / 692, accuracy:68.931 
Epoch: 5 Validation, loss: 0.59122, correct/total: 71 / 99, accuracy:71.717 
EarlyStopping counter: 1 out of 7
Epoch: 6 Training, loss: 0.56925, correct/total: 509 / 692, accuracy:73.555 
Epoch: 6 Validation, loss: 0.56050, correct/total: 66 / 99, accuracy:66.66

[I 2022-05-27 21:17:10,184] Trial 0 finished with value: 0.5238391077145934 and parameters: {'num_layer': 3, 'hidden_size': 41, 'learning_rate': 0.00675190785741219}. Best is trial 0 with value: 0.5238391077145934.


Epoch: 10 Validation, loss: 0.68542, correct/total: 58 / 98, accuracy:59.184 
EarlyStopping counter: 7 out of 7
Early stopping
Fold No. 1
Epoch: 1 Training, loss: 0.68578, correct/total: 391 / 692, accuracy:56.503 
Epoch: 1 Validation, loss: 0.65505, correct/total: 60 / 99, accuracy:60.606 
Epoch: 2 Training, loss: 0.67579, correct/total: 404 / 692, accuracy:58.382 
Epoch: 2 Validation, loss: 0.63712, correct/total: 60 / 99, accuracy:60.606 
Epoch: 3 Training, loss: 0.62957, correct/total: 457 / 692, accuracy:66.040 
Epoch: 3 Validation, loss: 0.51393, correct/total: 73 / 99, accuracy:73.737 
Epoch: 4 Training, loss: 0.62261, correct/total: 471 / 692, accuracy:68.064 
Epoch: 4 Validation, loss: 0.72821, correct/total: 57 / 99, accuracy:57.576 
EarlyStopping counter: 1 out of 7
Epoch: 5 Training, loss: 0.62301, correct/total: 455 / 692, accuracy:65.751 
Epoch: 5 Validation, loss: 0.50618, correct/total: 71 / 99, accuracy:71.717 
Epoch: 6 Training, loss: 0.60817, correct/total: 460 / 692

[I 2022-05-28 01:39:32,401] Trial 1 finished with value: 0.49272203748114407 and parameters: {'num_layer': 3, 'hidden_size': 51, 'learning_rate': 0.0012825376137445903}. Best is trial 1 with value: 0.49272203748114407.


Epoch: 22 Validation, loss: 0.57738, correct/total: 71 / 98, accuracy:72.449 
EarlyStopping counter: 7 out of 7
Early stopping
Fold No. 1
Epoch: 1 Training, loss: 0.67773, correct/total: 411 / 692, accuracy:59.393 
Epoch: 1 Validation, loss: 0.67404, correct/total: 52 / 99, accuracy:52.525 
Epoch: 2 Training, loss: 0.66951, correct/total: 430 / 692, accuracy:62.139 
Epoch: 2 Validation, loss: 0.68155, correct/total: 52 / 99, accuracy:52.525 
EarlyStopping counter: 1 out of 7
Epoch: 3 Training, loss: 0.65391, correct/total: 433 / 692, accuracy:62.572 
Epoch: 3 Validation, loss: 0.67190, correct/total: 62 / 99, accuracy:62.626 
Epoch: 4 Training, loss: 0.65080, correct/total: 453 / 692, accuracy:65.462 
Epoch: 4 Validation, loss: 0.79519, correct/total: 52 / 99, accuracy:52.525 
EarlyStopping counter: 1 out of 7
Epoch: 5 Training, loss: 0.67962, correct/total: 412 / 692, accuracy:59.538 
Epoch: 5 Validation, loss: 0.66964, correct/total: 52 / 99, accuracy:52.525 
Epoch: 6 Training, loss:

[I 2022-05-28 04:33:22,530] Trial 2 finished with value: 0.5742171974852681 and parameters: {'num_layer': 3, 'hidden_size': 121, 'learning_rate': 0.003942435469055516}. Best is trial 1 with value: 0.49272203748114407.


Epoch: 12 Validation, loss: 0.61065, correct/total: 74 / 98, accuracy:75.510 
EarlyStopping counter: 7 out of 7
Early stopping
Mejor resultado
[0.49272203748114407]
{'num_layer': 3, 'hidden_size': 51, 'learning_rate': 0.0012825376137445903}
